In [1]:
import math
import numpy as np
import pandas as pd
from scipy.stats import norm

df = pd.read_excel("../data/underlying_assets.xlsx", engine="openpyxl")

starting_date = "2017/06/30"
checking_date = np.array(["2017/12/21", "2018/06/26", "2018/12/20", "2019/06/25", "2019/12/19", "2020/06/25"])
strike_price = np.array([0.87, 0.87, 0.87, 0.85, 0.85, 0.85])

first_star_raw = df[df.Name == "삼성전자"].set_index("Date").loc[:, "Close"]
second_star_raw = df[df.Name == "현대차"].set_index("Date").loc[:, "Close"]

first_star_s0 = first_star_raw[starting_date]
second_star_s0 = second_star_raw[starting_date]

In [137]:
first_star_raw["2017-06-30"]

47540

In [73]:
def get_rebalancing_data(data):
    data.loc[:, 'year'] = data['Date'].dt.year
    data.loc[:, 'week_num'] = data['Date'].dt.week
    data.loc[:, 'dayofweek'] = data['Date'].dt.dayofweek

    data = data.loc[data.groupby(['year', 'week_num'])['dayofweek'].idxmin()].sort_values(by="Date").reset_index(drop=True)
    
    return data

first_star_rebalancing_data = get_rebalancing_data(df[df.Name == "삼성전자"])
second_star_rebalancing_data = get_rebalancing_data(df[df.Name == "현대차"])

/Users/jonghyun/miniforge3/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/var/folders/nw/x1lqw0rx1mj698bqcgqxhk880000gn/T/ipykernel_24791/6985285.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data.loc[:, 'week_num'] = data['Date'].dt.week


In [150]:
s0 = 49
k = 50
r = 0.05
mu = 0.13
sigma = 0.2
t = 0.3846

d1 = (math.log(s0 / k) + (r + math.pow(sigma, 2) / 2) * t) / (sigma * math.sqrt(t))
print(d1)
delta = norm.cdf(d1)



0.054173752978308974


In [142]:
def compute_parameters(data):
    mu = data.pct_change().mean() * math.sqrt(252)
    sigma = data.pct_change().std() * math.sqrt(252)
    
    return mu, sigma


def compute_delta(s0, k, r, t, mu, sigma):
    d1 = (math.log(s0 / k) + (r + math.pow(sigma, 2) / 2) * t) / (sigma * math.sqrt(t))
    delta = norm.cdf(d1) - 1
    
    return delta


def change_shares(old_amount, delta, price, face_amount=1000):
    new_amount = delta * face_amount
    change = round(new_amount - old_amount)
    
    cost = change * price
    
    return new_amount, cost

In [157]:
compute_parameters(second_star_raw)

(-0.00231279056756994, 0.31781953369010324)